## <center>Метод линеаризации для метрики CTR
* Считаем общий *CTR* в контрольной группе  $𝐶𝑇𝑅_{𝑐𝑜𝑛𝑡𝑟𝑜𝑙}=𝑠𝑢𝑚(𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(𝑣𝑖𝑒𝑤𝑠)$
* Посчитаем в обеих группах поюзерную метрику 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠 = 𝑙𝑖𝑘𝑒𝑠 − $𝐶𝑇𝑅_{𝑐𝑜𝑛𝑡𝑟𝑜𝑙}$ ∗ 𝑣𝑖𝑒𝑤𝑠
* После чего сравним t-тестом отличия в группах по метрике 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠.   

Сами по себе метрики отношения обычно лежат в границах от 0 до 1. Линеаризация позволяет расширить этот диапазон и перейти к большему размеру эффекта.
Обратите внимание, что дисперсия при таком подходе возрастает — но чувствительность всё равно увеличивается.   
Если тест на метрике linearized likes «прокрасится» и увидит изменения, значит изменения есть и в метрике исходной (то есть в лайках на пользователя и в пользовательских CTR).

### <center>Задача
1. Проанализируйте тест между группами 0 и 3 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?
2. Проанализируйте тест между группами 1 и 2 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?

In [59]:
import pandahouse as ph
import pandas as pd
from scipy import stats

In [2]:
connection = {'host':'https://clickhouse.lab.karpov.courses',
              'database':'simulator_20230120',
              'user':'student',
              'password':'dpo_python_2020'
             }

In [10]:
query = '''
select exp_group, 
    user_id, 
    sum(action = 'like') likes,
    sum(action = 'view') views,
    likes/views ctr
from {db}.feed_actions 
where toDate(time) between '2023-01-01' and '2023-01-07'
    and exp_group in (0, 1, 2, 3)
group by exp_group, user_id 
'''

In [11]:
df = ph.read_clickhouse(query, connection = connection)

In [13]:
df.head()

,exp_group,user_id,likes,views,ctr
0,3,115383,9,30,0.300000
1,1,18392,7,32,0.218750
2,2,131473,14,134,0.104478
3,3,123580,13,48,0.270833
4,2,32420,26,128,0.203125


### <center>Тест между группами 0 и 3.

In [46]:
df1 = df.query('exp_group==0 | exp_group==3').copy()

0 - контрольная группа, считаем для нее глобальный $𝐶𝑇𝑅_{control}$ :

In [48]:
ctr_control_0 = df1[df1['exp_group'] == 0].likes.sum() / df1[df1['exp_group'] == 0].views.sum()

Посчитаем в обеих группах поюзерную метрику 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠 и добавим в датафрейм:

In [49]:
linearized_likes_0 = df1[df1['exp_group'] == 0].likes - ctr_control_0 * df1[df1['exp_group'] == 0].views
linearized_likes_3 = df1[df1['exp_group'] == 3].likes - ctr_control_0 * df1[df1['exp_group'] == 3].views

In [51]:
df1.loc[df1['exp_group'] == 0,'linearized_likes'] = linearized_likes_0
df1.loc[df1['exp_group'] == 3,'linearized_likes'] = linearized_likes_3

In [52]:
df1.head()

,exp_group,user_id,likes,views,ctr,linearized_likes
0,3,115383,9,30,0.300000,2.752916
3,3,123580,13,48,0.270833,3.004666
6,0,30319,21,110,0.190909,-1.905973
8,3,118359,7,38,0.184211,-0.912973
10,0,7626,14,39,0.358974,5.878791


Выполним тесты Стьюдента по метрикам CTR и метрике линеаризованных лайков:

In [53]:
stats.ttest_ind(df1[df1['exp_group']==0].ctr, 
                df1[df1['exp_group']==3].ctr,
                equal_var=False)

Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)

In [54]:
stats.ttest_ind(df1[df1['exp_group']==0].linearized_likes, 
                df1[df1['exp_group']==3].linearized_likes,
                equal_var=False)

Ttest_indResult(statistic=-15.214995460903827, pvalue=5.4914249479690016e-52)

 ### Выводы:
 - Оба теста по обеим метрикам показали статистически значимые различия между контрольной (0) и тестовой (3) группами.
 - p-value по метрике 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠 стало меньше по сравнению с метрикой CTR.

### <center>Тест между группами 1 и 2.

In [35]:
df2 = df.query('exp_group==1 | exp_group==2').copy()

In [36]:
df.head()

,exp_group,user_id,likes,views,ctr
0,3,115383,9,30,0.300000
1,1,18392,7,32,0.218750
2,2,131473,14,134,0.104478
3,3,123580,13,48,0.270833
4,2,32420,26,128,0.203125


1 - контрольная группа, считаем для нее глобальный $𝐶𝑇𝑅_{control}$ :

In [56]:
ctr_control_1 = df2[df2['exp_group'] == 1].likes.sum() / df2[df2['exp_group'] == 1].views.sum()

Посчитаем в обеих группах поюзерную метрику 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠 и добавим в датафрейм:

In [58]:
linearized_likes_1 = df2[df2['exp_group'] == 1].likes - ctr_control_1 * df2[df2['exp_group'] == 1].views
linearized_likes_2 = df2[df2['exp_group'] == 2].likes - ctr_control_1 * df2[df2['exp_group'] == 2].views

In [61]:
df2.loc[df2['exp_group'] == 1,'linearized_likes'] = linearized_likes_1
df2.loc[df2['exp_group'] == 2,'linearized_likes'] = linearized_likes_2

In [64]:
df2.head()

,exp_group,user_id,likes,views,ctr,linearized_likes
1,1,18392,7,32,0.218750,0.343142
2,2,131473,14,134,0.104478,-13.875592
4,2,32420,26,128,0.203125,-0.627431
5,2,24239,40,90,0.444444,21.277588
7,1,27443,2,29,0.068966,-4.032777


Выполним тесты Стьюдента по метрикам CTR и метрике линеаризованных лайков:

In [66]:
stats.ttest_ind(df2[df2['exp_group']==1].ctr, 
                df2[df2['exp_group']==2].ctr,
                equal_var=False)

Ttest_indResult(statistic=0.7094392041270485, pvalue=0.4780623130874935)

In [67]:
stats.ttest_ind(df2[df2['exp_group']==1].linearized_likes, 
                df2[df2['exp_group']==2].linearized_likes,
                equal_var=False)

Ttest_indResult(statistic=6.122579994775974, pvalue=9.439432187037644e-10)

 ### Выводы:
 - Только тест по метрике линеаризованных лайков показали статистически значимые различия между контрольной (0) и тестовой (3) группами.
 - p-value по метрике 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠 стало на порядки меньше по сравнению с метрикой CTR.

Эмперически доказано что метод линеаризации метрики позволяет значительно увеличить чувствительность t критерия.  И при этом гарантируется (в отличие сглаживания Лапласа), что если тест на этой другой метрике «прокрасится» и увидит изменения, значит изменения есть и в метрике исходной (то есть в лайках на пользователя и в пользовательских CTR)